Virtual Acoustics and Immersive Audio Workshop - CCRMA Stanford University  
31.07.25 - Orchisama Das, Gloria Dal Santo
  
### L09: Spatial Impulse Response Rendering

In this assignment we will 
- Read FOA SRIRs for a single source-receiver pair in the racquetball court and run SIRR analysis on it.
- Output RIR files for 22.1 speaker setup in studio E.
- Listen to auralization with `sparta_multiconv` plugin that does multichannel convolution of the dry signal with each of the synthesized RIRs for the loudspeakers.

In [ ]:
import numpy as np
import soundfile as sf
import json
from pathlib import Path
from scipy.io import loadmat
import spaudiopy as spa
from numpy.typing import ArrayLike, NDArray
from spatial_audio.sirr import SIRRParameters, SIRR
from utils import ms_to_samps

### Part 1
Complete the functions `calculate_parameters()`, `process_directional_part()` and `process_frame()`,  in the `SIRR` class. 
- `calculate_parameters()` should return an object of dataclass `SIRRParameters`. These parameters are then smoothed over time to prevent DoA smearing and phasiness. This is done in the `process()` function. The smoothed parameters are saved in an `SIRRParameters` dataclass called `self.smoothed_parameters`.
- `process_directional_part` takes in as input the directional part of the current time-frame. It should use the smoothed DoAs to calculate the VBAP gains for each frequency-bin and each loudspeaker, and return the directional components for each frequency bin and all 22 loudspeakers. The VBAP implementation has been provided in `spatial.py`.
- `process_frame()` should return the output signal for the current time frame (of shape `num_loudspeakers, num_frequency_bins`) by decomposing `cur_stft_frame` (of shape `num_ambi_channels, num_frequency_bins`) into direct and diffuse parts, processing them separately and adding them.

Read an SRIR you have synthesized in Lecture 7's assignment and generate the loudspeaker signals obtained from SIRR for playback in CCRMA's studio E. Save the first 1.5s of the 22-channel output file. **We are saving 1.5s because my laptop can do 22 simultaneous convolutions in real-time only with such a short signal**.

In [ ]:
# Read studio E speaker positions in spherical coordinates

ls_layout_path = Path('../data/Week 2/CCRMAStudioEFlippedAzimuth.json').resolve()
with open(ls_layout_path) as json_file:
    json_data = json.load(json_file)
    ls_layout = json_data['GenericLayout']['Elements']
    num_ls = len(ls_layout)
    ls_dirs = np.zeros((num_ls, 3))
    for k, ls in enumerate(ls_layout):
        ls_dirs[k, 0] = ls['Azimuth']
        ls_dirs[k, 1] = ls['Elevation']
        ls_dirs[k, 2] = ls['Radius']

In [ ]:
# Read B format RIR from source position I and receiver position A
src = 'I'
rec = 'A'
srir_path = Path(f'../data/Week 2/shoe-box-synth-comp/ambi_rec/Bformat_Speaker_src={src}_rec={rec}.wav').resolve()
srir, fs = sf.read(str(srir_path))

# Truncate RIR to 1.5s
trunc_len_ms = 1500
trunc_len_samps = ms_to_samps(trunc_len_ms, fs)
srir_trunc = srir[:trunc_len_samps, :]

In [ ]:
#### WRITE YOUR CODE HERE ####

# Create SIRR object
sirr_obj = SIRR(srir_trunc, fs, ls_dirs, 'spherical')

# Get output_signal by calling SIRR's process() function
output_signal = sirr_obj.process()

# Save the SIRR synthesised RIRs for each speaker in Studio E
save_path = Path(f'../data/Week 2/shoe-box-synth-comp/sirr/SIRR_Bformat_Speaker_src={src}_rec={rec}_trunc.wav').resolve()
sf.write(str(save_path), output_signal.T, fs)

### Part 2

Log the DoAs of each time-frame and for each frequency bin by making use of `DataLogger` in `utils.py`. Save the DoAs in a mat file with the `save_history` function. Load the mat file and plot the DoAs for the 1kHz frequency bin for each time-frame. You can use `spaudiopy`'s `plot.doa` function

In [ ]:
def plot_doa(freq_bins: ArrayLike, az: ArrayLike, el: NDArray, fs: float, freq_to_plot: float = 1000):
    """Plot the DOA estimates over several time frames for frequency = freq_to_plot Hz"""

    # find the closest frequency to freq_to_plot
    closest_idx = np.argmin(np.abs(freq_bins - freq_to_plot))

    # get azimuth and elevation angles
    cur_az = az[:, closest_idx]
    cur_el = el[:, closest_idx]
    cur_r = np.ones_like(freq_bins)
    ps = ps = 1 / np.exp(np.linspace(0, 3, len(cur_az)))

    # call spaaudiopy's plot.doa function
    spa.plot.doa(cur_az, np.pi / 2 - cur_el, ps, fs=fs, ltitle=f'{freq_to_plot}Hz')

In [ ]:
log_path = Path('../data/Week 2/shoe-box-synth-comp/sirr/').resolve()

# Save the estimated DoAs by calling save_history in SIRR
sirr_obj.save_history(log_path)

# Load the saved DoAs
data_log = loadmat(f'{log_path}/history.mat')
az = data_log['azimuth']
el = data_log['elevation']

# Plot the saved DoAs
plot_doa(sirr_obj.freqs, az, el, fs=fs)